In [ ]:
import hist
import matplotlib.gridspec as grid_spec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
from iminuit import Minuit, cost
from scipy.stats import t
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split

plt.style.use(["science", "notebook"])

In [ ]:
plt.rcParams["font.size"] = 14
plt.rcParams["axes.formatter.limits"] = -5, 4
plt.rcParams["figure.figsize"] = 6, 4
colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]

In [ ]:
df = pd.read_csv("features_stable.csv")

In [ ]:
start_z = df.pop("start_z").values
nu_energy = np.log(df.pop("nu_energy").values)

In [ ]:
X = df.values

In [ ]:
z_min = -232.3746
z_max = -82.3862

In [ ]:
X_train, X_test, start_z_train, start_z_test = train_test_split(
    X, start_z, random_state=0
)

In [ ]:
estimate_start_z = GradientBoostingRegressor(random_state=0)

estimate_start_z.fit(X_train, start_z_train)

In [ ]:
# TODO use adaboost and hyperoptimise?

In [ ]:
start_z_pred = estimate_start_z.predict(X_test)

estimate_start_z.score(X_test, start_z_test)

In [ ]:
h = hist.Hist.new.Regular(10, -10, +10, name=r"𝛥z [cm]").Double()

In [ ]:
h.fill(start_z_pred - start_z_test)

In [ ]:
def model(x, mu, sigma):
    return scipy.stats.norm.cdf(x, mu, sigma)

In [ ]:
entries, edges = h.to_numpy()

In [ ]:
m = Minuit(cost.BinnedNLL(entries, edges, model), 0, 25)

In [ ]:
res = m.migrad()

In [ ]:
h.plot()
plt.xlabel(r"$\Delta z\;[\mathrm{cm}]$")
plot_range = -10, 10
x = np.linspace(*plot_range, 100)
best_fit = scipy.stats.norm(res.params[0].value, res.params[1].value)
# best_fit = scipy.stats.norm(0.044, 2.83) # TODO take from fit
n_bins = 10
binsize = (plot_range[1] - plot_range[0]) / n_bins
scale = h.sum() / (best_fit.cdf(plot_range[1]) - best_fit.cdf(plot_range[0])) * binsize
plt.plot(x, scale * best_fit.pdf(x))
ax = plt.gca()
# plt.text(0.6, 0.9, r"$\mu = 0.044 $\;cm", transform=ax.transAxes, usetex=True)
plt.text(
    0.6,
    0.9,
    rf"$\mu = {res.params[0].value:.2f} \pm {res.params[0].error:.2f}$\;cm",
    transform=ax.transAxes,
    usetex=True,
)
# plt.text(0.6, 0.81, r"$\sigma = 2.83 $\;cm", transform=ax.transAxes, usetex=True)
plt.text(
    0.6,
    0.81,
    rf"$\sigma = {res.params[1].value:.2f} \pm {res.params[1].error:.2f}$\;cm",
    transform=ax.transAxes,
    usetex=True,
)
plt.text(
    0.0,
    1.02,
    "preliminary",
    fontfamily="sans-serif",
    fontsize=16,
    transform=ax.transAxes,
    usetex=False,
)
plt.text(
    0.8,
    1.02,
    "AdvSND",
    fontweight="bold",
    fontfamily="sans-serif",
    fontsize=16,
    transform=ax.transAxes,
    usetex=False,
)
plt.savefig("plots/h_dz.pdf")
plt.savefig("plots/h_dz.png")

In [ ]:
X_train, X_test, nu_energy_train, nu_energy_test = train_test_split(
    X, nu_energy, random_state=0
)

In [ ]:
estimate_nu_energy = AdaBoostRegressor(
    random_state=0, learning_rate=0.1, n_estimators=466
)

estimate_nu_energy.fit(X_train, nu_energy_train)

In [ ]:
"""
bdt_opt = BayesSearchCV(
    AdaBoostRegressor(),
    {
    'learning_rate': Real(0.005, 0.9, prior="log-uniform"),
    'n_estimators': Integer(1, 1000),
},
    n_iter=100,
    cv=5
)

bdt_opt.fit(X_train, nu_energy_train)
"""

In [ ]:
"""
bdt_opt.best_params_
"""

In [ ]:
nu_energy_pred = estimate_nu_energy.predict(X_test)

estimate_nu_energy.score(X_test, nu_energy_test)

In [ ]:
E = np.exp(nu_energy_test)

In [ ]:
E_scale_correction = np.mean(E) - np.mean(np.exp(nu_energy_pred))

In [ ]:
dE_over_E = (np.exp(nu_energy_pred) - E) / E

In [ ]:
dE_over_E_corrected = (
    (np.exp(nu_energy_pred) + E_scale_correction) - np.exp(nu_energy_test)
) / E

In [ ]:
h_nu_energy = hist.Hist.new.Regular(50, -5, +5, name=r"𝛥E/E").Double()

In [ ]:
h_nu_energy_corrected = hist.Hist.new.Regular(50, -5, +5, name=r"𝛥E/E").Double()

In [ ]:
h_nu_energy.fill(dE_over_E)
h_nu_energy_corrected.fill(dE_over_E_corrected)

In [ ]:
h_nu_energy.plot(label="AdaBoost")
# h_nu_energy_corrected.plot(label="AdaBoost + correction of average")
plt.xlabel(r"$\frac{\Delta E}{E}$")
# plt.legend()
ax = plt.gca()
plt.text(
    0.0,
    1.02,
    "preliminary",
    fontfamily="sans-serif",
    fontsize=16,
    transform=ax.transAxes,
    usetex=False,
)
plt.text(
    0.8,
    1.02,
    "AdvSND",
    fontweight="bold",
    fontfamily="sans-serif",
    fontsize=16,
    transform=ax.transAxes,
    usetex=False,
)
plt.savefig("plots/h_dE_over_E.png")
plt.savefig("plots/h_dE_over_E.pdf")

In [ ]:
h_dE_over_E_vs_E = (
    hist.Hist.new.Regular(50, 0, 5000, name=r"E")
    .Regular(50, -1, +1, name=r"𝛥E/E")
    .Double()
)

In [ ]:
h_dE_over_E_vs_E.fill(E, dE_over_E)

In [ ]:
h_dE_over_E_vs_E_corrected = (
    hist.Hist.new.Regular(50, 0, 5000, name=r"E")
    .Regular(50, -1, +1, name=r"𝛥E/E")
    .Double()
)

In [ ]:
h_dE_over_E_vs_E_corrected.fill(E, dE_over_E_corrected)

In [ ]:
sorted_E = E.copy()
sorted_E.sort()

In [ ]:
# h_dE_over_E_vs_E.plot()
h_dE_over_E_vs_E_corrected.plot()
plt.plot(sorted_E, E_scale_correction / sorted_E, label="Scale correction", color="red")
ax = plt.gca()
plt.text(
    0.8,
    1.02,
    "AdvSND",
    fontweight="bold",
    fontfamily="sans-serif",
    fontsize=16,
    transform=ax.transAxes,
    usetex=False,
)
plt.xlabel(r"true $E\;[\mathrm{GeV}]$")
plt.ylabel(r"corrected $\frac{\Delta E}{E}$")
plt.legend(loc="upper left", bbox_to_anchor=(0.2, 1.13))
plt.text(
    0.0,
    1.02,
    "prelim.",
    fontfamily="sans-serif",
    fontsize=16,
    transform=ax.transAxes,
    usetex=False,
)
plt.savefig("plots/h_dE_over_E_vs_E_corrected.png")
plt.savefig("plots/h_dE_over_E_vs_E_corrected.pdf")

In [ ]:
h_dE_over_E_vs_E.plot()
ax = plt.gca()
plt.text(
    0.8,
    1.02,
    "AdvSND",
    fontweight="bold",
    fontfamily="sans-serif",
    fontsize=16,
    transform=ax.transAxes,
    usetex=False,
)
plt.xlabel(r"true $E\;[\mathrm{GeV}]$")
plt.ylabel(r"$\frac{\Delta E}{E}$")
# plt.legend(loc='upper left', bbox_to_anchor=(0.2, 1.13))
plt.text(
    0.0,
    1.02,
    "preliminary",
    fontfamily="sans-serif",
    fontsize=16,
    transform=ax.transAxes,
    usetex=False,
)
plt.savefig("plots/h_dE_over_E_vs_E.png")
plt.savefig("plots/h_dE_over_E_vs_E.pdf")

In [ ]:
h_dE_over_E_vs_E_reco = (
    hist.Hist.new.Regular(
        15, 80, 2500, name=r"E_reco", transform=hist.axis.transform.log
    )
    .Regular(50, -1, +5, name=r"𝛥E/E")
    .Double()
)

In [ ]:
h_dE_over_E_vs_E_reco.fill(np.exp(nu_energy_pred), dE_over_E)

In [ ]:
h_dE_over_E_vs_E_reco.plot()
plt.xlabel(r"reconstructed $E\;[\mathrm{GeV}]$")
plt.ylabel(r"$\frac{\Delta E}{E}$")
ax = plt.gca()
plt.text(
    0.8,
    1.02,
    "AdvSND",
    fontweight="bold",
    fontfamily="sans-serif",
    fontsize=16,
    transform=ax.transAxes,
    usetex=False,
)
plt.text(
    0.0,
    1.02,
    "preliminary",
    fontfamily="sans-serif",
    fontsize=16,
    transform=ax.transAxes,
    usetex=False,
)
plt.savefig("plots/h_dE_over_E_vs_E_reco.png")
plt.savefig("plots/h_dE_over_E_vs_E_reco.pdf")

In [ ]:
np.mean(
    h_dE_over_E_vs_E_reco.values(), axis=1, where=h_dE_over_E_vs_E_reco.values() != 0
)

In [ ]:
nu_energy_pred[0]  # TODO how to get bin?

In [ ]:
plt.errorbar(
    x=np.logspace(4.5, 8, 15, base=np.e),
    y=np.mean(
        h_dE_over_E_vs_E_reco.values(),
        axis=1,
        where=h_dE_over_E_vs_E_reco.values() != 0,
    ),
    yerr=np.std(
        h_dE_over_E_vs_E_reco.values(),
        axis=1,
        where=h_dE_over_E_vs_E_reco.values() != 0,
    ),
    linestyle="none",
    marker="o",
)

ax = plt.gca()
plt.text(
    0.1,
    0.5,
    "VERY PRELIMINARY",
    fontfamily="sans-serif",
    fontsize=30,
    transform=ax.transAxes,
    usetex=False,
    color="gray",
    zorder=0,
    alpha=0.5,
)
plt.text(
    0.8,
    1.02,
    "AdvSND",
    fontweight="bold",
    fontfamily="sans-serif",
    fontsize=16,
    transform=ax.transAxes,
    usetex=False,
)
# plt.plot(np.logspace(4.5, 8, 15, base=np.e), res.intercept + res.slope*np.linspace(np.exp(4.5), np.exp(8), 15), 'r', label='fitted line')
plt.xlabel(r"reconstructed $E\;[\mathrm{GeV}]$")
plt.ylabel(r"$\left<\frac{\Delta E}{E}\right>$")
plt.savefig("plots/non_const_correction.png")
plt.savefig("plots/non_const_correction.pdf")
# TODO y-axis makes no sense!

In [ ]:
res = scipy.stats.linregress(
    np.linspace(np.exp(4.5), np.exp(8), 15),
    np.mean(
        h_dE_over_E_vs_E_reco.values(),
        axis=1,
        where=h_dE_over_E_vs_E_reco.values() != 0,
    ),
)

In [ ]:
# Two-sided inverse Students t-distribution

# p - probability, df - degrees of freedom


def tinv(p, df):
    return abs(t.ppf(p / 2, df))

In [ ]:
ts = tinv(0.05, 15 - 2)

print(f"slope (95%): {res.slope:.6f} +/- {ts * res.stderr:.6f}")

print(f"intercept (95%): {res.intercept:.6f} +/- {ts * res.intercept_stderr:.6f}")

In [ ]:
def E_true_vs_E_reco(E_true, E_reco, plot_name="h_E_true_vs_E_reco"):
    h_E_true_vs_E_reco = (
        hist.Hist.new.Regular(100, 0, 5000, name=r"E_true")
        .Regular(100, 0, 5000, name=r"E_reco")
        .Double()
    )
    h_E_true_vs_E_reco.fill(E_true, E_reco)
    h_E_true_vs_E_reco.plot()
    plt.plot(sorted_E, sorted_E)
    plt.xlabel(r"true $E\;[\mathrm{GeV}]$")
    plt.ylabel(r"reconstructed $E\;[\mathrm{GeV}]$")
    ax = plt.gca()
    plt.text(
        0.8,
        1.02,
        "AdvSND",
        fontweight="bold",
        fontfamily="sans-serif",
        fontsize=16,
        transform=ax.transAxes,
        usetex=False,
    )
    plt.text(
        0.0,
        1.02,
        "preliminary",
        fontfamily="sans-serif",
        fontsize=16,
        transform=ax.transAxes,
        usetex=False,
    )
    plt.savefig(f"plots/{plot_name}.png")
    plt.savefig(f"plots/{plot_name}.pdf")

In [ ]:
# TODO average percentage difference per bin? Use to correct?

In [ ]:
E_true_vs_E_reco(E, np.exp(nu_energy_pred))

# Is the position independent of energy?

In [ ]:
h_dE_over_E_vs_z = (
    hist.Hist.new.Regular(50, -200, -80, name=r"z")
    .Regular(50, -1, +4, name=r"𝛥E/E")
    .Double()
)

In [ ]:
h_dE_over_E_vs_z.fill(start_z_test, dE_over_E)

In [ ]:
h_dE_over_E_vs_dz = (
    hist.Hist.new.Regular(50, -10, 10, name=r"z")
    .Regular(50, -1, +4, name=r"𝛥E/E")
    .Double()
)

In [ ]:
h_dE_over_E_vs_dz.fill(start_z_test - start_z_pred, dE_over_E)

In [ ]:
h_dE_over_E_vs_dz.plot()
plt.ylabel(r"$\frac{\Delta E}{E}$")
plt.xlabel(r"$\Delta z\;[\mathrm{cm}]$")
ax = plt.gca()
plt.text(
    0.8,
    1.02,
    "AdvSND",
    fontweight="bold",
    fontfamily="sans-serif",
    fontsize=16,
    transform=ax.transAxes,
    usetex=False,
)
plt.text(
    0.0,
    1.02,
    "preliminary",
    fontfamily="sans-serif",
    fontsize=16,
    transform=ax.transAxes,
    usetex=False,
)
plt.savefig("plots/h_dE_over_E_vs_dz.pdf")
plt.savefig("plots/h_dE_over_E_vs_dz.png")

In [ ]:
h_dE_over_E_vs_z.plot()
plt.ylabel(r"$\frac{\Delta E}{E}$")
plt.xlabel(r"$z\;[\mathrm{cm}]$")
ax = plt.gca()
plt.text(
    0.8,
    1.02,
    "AdvSND",
    fontweight="bold",
    fontfamily="sans-serif",
    fontsize=16,
    transform=ax.transAxes,
    usetex=False,
)
plt.text(
    0.0,
    1.02,
    "preliminary",
    fontfamily="sans-serif",
    fontsize=16,
    transform=ax.transAxes,
    usetex=False,
)
plt.savefig("plots/h_dE_over_E_vs_z.pdf")
plt.savefig("plots/h_dE_over_E_vs_z.png")

Position resolution seems independent of energy, can smear true position using position resolution.

## Smear start by gaussian and retrain energy regressions

In [ ]:
scipy.stats.norm(
    loc=np.mean(start_z_test - start_z_pred), scale=np.std(start_z_test - start_z_pred)
).rvs(10000)  # TODO update with values from fit

In [ ]:
df["z_smeared"] = start_z + scipy.stats.norm(
    loc=np.mean(start_z_test - start_z_pred), scale=np.std(start_z_test - start_z_pred)
).rvs(10000)

In [ ]:
X = df.values

In [ ]:
X_train, X_test, nu_energy_train, nu_energy_test = train_test_split(
    X, nu_energy, random_state=0
)

In [ ]:
"""
bdt_opt_smeared = BayesSearchCV(
    AdaBoostRegressor(),
    {
    'learning_rate': Real(0.005, 0.9, prior="log-uniform"),
    'n_estimators': Integer(1, 1000),
},
    n_iter=20,
    cv=3
)

bdt_opt_smeared.fit(X_train[:1000], nu_energy_train[:1000])
"""

In [ ]:
"""
bdt_opt_smeared.best_params_
"""

In [ ]:
estimate_nu_energy_with_smeared_z = AdaBoostRegressor(
    random_state=0, learning_rate=0.08, n_estimators=610
)

estimate_nu_energy_with_smeared_z.fit(X_train, nu_energy_train)

In [ ]:
nu_energy_pred = estimate_nu_energy_with_smeared_z.predict(X_test)

estimate_nu_energy_with_smeared_z.score(X_test, nu_energy_test)

In [ ]:
plt.hist(np.exp(nu_energy_test) - np.exp(nu_energy_pred))

* Shouldn't have expected improvement just because of smeared (or even *true* z).
* Important to translate strips per station by the start station (how to deal with HCAL?)

In [ ]:
plt.hist(start_z - min(start_z))

In [ ]:
df["smeared_start_station"] = np.clip(
    ((df.z_smeared - (-232.3746)) / 1.5).astype(int), 0, 99
)

In [ ]:
absolute_strip_counts = df[[f"target_n_hits_station_{n}" for n in range(100)]].values

In [ ]:
A = absolute_strip_counts

In [ ]:
r = -df.smeared_start_station.values

In [ ]:
# https://stackoverflow.com/a/20361561

rows, column_indices = np.ogrid[: A.shape[0], : A.shape[1]]

# Always use a negative shift, so that column_indices are valid.
# Alternative: r %= A.shape[1]
r[r < 0] += A.shape[1]
column_indices = column_indices - r[:, np.newaxis]

result = A[rows, column_indices]

In [ ]:
X_relative = result

In [ ]:
X_relative = np.concatenate([X[:, 1:-2], result], axis=1)

In [ ]:
X_rel_train, X_rel_test, log_E_rel_train, log_E_rel_test = train_test_split(
    X_relative, nu_energy, random_state=0
)

In [ ]:
estimate_nu_energy_rel = AdaBoostRegressor(
    random_state=0, learning_rate=0.1, n_estimators=500
)
estimate_nu_energy_rel.fit(X_rel_train, log_E_rel_train)

In [ ]:
log_E_rel_pred = estimate_nu_energy_rel.predict(X_rel_test)
estimate_nu_energy_rel.score(X_rel_test, log_E_rel_test)

In [ ]:
# TODO missing global info and HCAL info!
# TODO how to make the HCAL info relative to position?

In [ ]:
E_rel_pred = np.exp(log_E_rel_pred)
E_rel_test = np.exp(log_E_rel_test)

In [ ]:
dE_rel = E_rel_pred - E_rel_test

In [ ]:
dE_over_E_rel = dE_rel / E_rel_test

In [ ]:
h_dE_over_E_rel_vs_E_rel_test = (
    hist.Hist.new.Regular(50, 0, 5000, name=r"E")
    .Regular(50, -1, +1, name=r"𝛥E/E")
    .Double()
)

In [ ]:
h_dE_over_E_rel_vs_E_rel_test.fill(E_rel_test, dE_over_E_rel)

In [ ]:
h_dE_over_E_rel_vs_E_rel_test.plot()
ax = plt.gca()
plt.text(
    0.8,
    1.02,
    "AdvSND",
    fontweight="bold",
    fontfamily="sans-serif",
    fontsize=16,
    transform=ax.transAxes,
    usetex=False,
)
plt.xlabel(r"true $E\;[\mathrm{GeV}]$")
plt.ylabel(r"$\frac{\Delta E}{E}$")
# plt.legend(loc='upper left', bbox_to_anchor=(0.2, 1.13))
plt.text(
    0.0,
    1.02,
    "preliminary",
    fontfamily="sans-serif",
    fontsize=16,
    transform=ax.transAxes,
    usetex=False,
)
plt.savefig("plots/h_dE_over_E_rel_vs_E_rel_test.pdf")
plt.savefig("plots/h_dE_over_E_rel_vs_E_rel_test.png")

In [ ]:
E_true_vs_E_reco(E_rel_test, E_rel_pred, plot_name="h_E_rel_test_vs_E_rel_pred")

In [ ]:
h_dE_over_E_rel = hist.Hist.new.Regular(50, -5, +5, name=r"𝛥E/E").Double()

In [ ]:
h_dE_over_E_rel.fill(dE_over_E_rel)

In [ ]:
# h_nu_energy.plot(label="BDT")
# h_nu_energy_corrected.plot(label="+correction of average")
h_dE_over_E_rel.plot(label="+relative ECAL info")
plt.xlabel(r"$\frac{\Delta E}{E}$")
# plt.legend()
ax = plt.gca()
plt.text(
    0.0,
    1.02,
    "preliminary",
    fontfamily="sans-serif",
    fontsize=16,
    transform=ax.transAxes,
    usetex=False,
)
plt.text(
    0.8,
    1.02,
    "AdvSND",
    fontweight="bold",
    fontfamily="sans-serif",
    fontsize=16,
    transform=ax.transAxes,
    usetex=False,
)

In [ ]:
bins_E_reco = 15

In [ ]:
h_dE_rel_test_vs_E_rel_pred = (
    hist.Hist.new.Regular(100, -3150, 950, name=r"dE")
    .Regular(bins_E_reco, 60, 2200, name=r"E_reco", transform=hist.axis.transform.log)
    .Double()
)

In [ ]:
h_dE_rel_test_vs_E_rel_pred.fill(dE_rel, E_rel_pred)

In [ ]:
h_dE_rel_test_vs_E_rel_pred.plot()
plt.xlabel(r" $\Delta E\;[\mathrm{GeV}]$")
plt.ylabel(r"reconstructed $E\;[\mathrm{GeV}]$")
ax = plt.gca()
plt.text(
    0.8,
    1.02,
    "AdvSND",
    fontweight="bold",
    fontfamily="sans-serif",
    fontsize=16,
    transform=ax.transAxes,
    usetex=False,
)
plt.text(
    0.0,
    1.02,
    "preliminary",
    fontfamily="sans-serif",
    fontsize=16,
    transform=ax.transAxes,
    usetex=False,
)
plt.savefig("plots/h_dE_rel_test_vs_E_rel_pred.pdf")
plt.savefig("plots/h_dE_rel_test_vs_E_rel_pred.png")

In [ ]:
def model(x, mu, sigma):
    return scipy.stats.norm.cdf(x, loc=mu, scale=sigma)

In [ ]:
gs = grid_spec.GridSpec(bins_E_reco, 1)
fig = plt.figure(figsize=(16, 9))

i = 0
mus = []
sigmas = []
bins = []

ax_objs = []
for bin in range(bins_E_reco):
    # country = countries[i]
    # = np.array(data[data.country == country].score)
    # x_d = np.linspace(0,1, 1000)

    # creating new axes object
    ax_objs.append(fig.add_subplot(gs[i : i + 1, 0:]))

    # plotting the distribution
    h = h_dE_rel_test_vs_E_rel_pred[:, bin]
    h.plot(yerr=False, ax=ax_objs[-1], color=colors[bin % len(colors)], histtype="fill")
    entries, edges = h.to_numpy()
    n_bins = len(entries)
    m = Minuit(cost.BinnedNLL(entries, edges, model), 0, 25)
    res = m.migrad()
    if res.valid:
        plot_range = ax_objs[-1].get_xlim()
        x = np.linspace(*plot_range, 100)
        best_fit = scipy.stats.norm(res.params[0].value, res.params[1].value)
        binsize = (plot_range[1] - plot_range[0]) / n_bins
        scale = (
            h.sum()
            / (best_fit.cdf(plot_range[1]) - best_fit.cdf(plot_range[0]))
            * binsize
        )
        ax_objs[-1].plot(
            x, scale * best_fit.pdf(x), color=colors[(bin + 3) % len(colors)]
        )
        bins.append(bin)
        mus.append(res.params[0])
        sigmas.append(res.params[1])
    else:
        print(res)

    # setting uniform x and y lims
    # ax_objs[-1].set_ylim(0,2.5)

    # make background transparent
    rect = ax_objs[-1].patch
    rect.set_alpha(0)

    # remove borders, axis ticks, and labels
    ax_objs[-1].set_yticklabels([])

    if i == bins_E_reco - 1:
        ax_objs[-1].set_xlabel(r"$\Delta E$", fontsize=16, fontweight="bold")
    else:
        ax_objs[-1].set_xticklabels([])
        ax_objs[-1].set_xlabel("")

    ax_objs[-1].set_ylabel(str(bin), rotation=45)
    ax_objs[-1].set_yticks([])
    ax_objs[-1].set_xticks([])

    spines = ["top", "right", "left", "bottom"]
    for s in spines:
        ax_objs[-1].spines[s].set_visible(False)

    #  ax_objs[-1].text(-0.02,0,adj_country,fontweight="bold",fontsize=14,ha="right")

    i += 1

gs.update(hspace=-0.7)
# gs.update()

plt.tight_layout()
plt.show()

In [ ]:
bin_edges = h_dE_rel_test_vs_E_rel_pred[0, :].to_numpy()[1]
bin_centres = (bin_edges[1:] + bin_edges[:-1]) / 2
bin_half_widths = (bin_edges[1:] - bin_edges[:-1]) / 2

In [ ]:
fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.set_xlabel("X data")
ax1.set_ylabel("Y1 data", color="g")
ax2.set_ylabel("Y2 data", color="b")
ax1.errorbar(
    bin_centres[bins],
    [mu.value for mu in mus] / bin_centres[bins],
    xerr=bin_half_widths[bins],
    yerr=[mu.error for mu in mus] / bin_centres[bins],
    linestyle="",
    label=r"$\left<\Delta E\right>$",
    color=colors[0],
)
ax2.errorbar(
    bin_centres[bins],
    [sigma.value for sigma in sigmas] / bin_centres[bins],
    xerr=bin_half_widths[bins],
    yerr=[sigma.error for sigma in sigmas] / bin_centres[bins],
    linestyle="",
    label=r"$\sigma\left(\Delta E\right)$",
    color=colors[1],
)
# ax1.hlines(0, *plt.xlim(), color='red')
ax1.set_ylabel(r"$\frac{\left<\Delta E\right>}{E_\mathrm{reco}}$", color=colors[0])
ax2.set_ylabel(
    r"$\frac{\sigma\left(\Delta E\right)}{E_\mathrm{reco}}$", color=colors[1]
)
ax1.set_xlabel(r"$E_\mathrm{reco}\;[\mathrm{GeV}]$")

In [ ]:
plt.errorbar(
    bin_centres[bins],
    [mu.value for mu in mus] / bin_centres[bins],
    xerr=bin_half_widths[bins],
    yerr=[mu.error for mu in mus] / bin_centres[bins],
    linestyle="",
    label=r"$\left<\Delta E\right>$",
    color=colors[0],
)
plt.hlines(0, *plt.xlim(), color="red")
plt.ylabel(r"$\frac{\left<\Delta E\right>}{E_\mathrm{reco}}$")
plt.xlabel(r"$E_\mathrm{reco}\;[\mathrm{GeV}]$")
ax = plt.gca()
plt.text(
    0.0,
    1.02,
    "preliminary",
    fontfamily="sans-serif",
    fontsize=16,
    transform=ax.transAxes,
    usetex=False,
)
plt.text(
    0.8,
    1.02,
    "AdvSND",
    fontweight="bold",
    fontfamily="sans-serif",
    fontsize=16,
    transform=ax.transAxes,
    usetex=False,
)
plt.savefig("plots/energy_bias.pdf")
plt.savefig("plots/energy_bias.png")

In [ ]:
plt.errorbar(
    bin_centres[bins],
    [sigma.value for sigma in sigmas] / bin_centres[bins],
    xerr=bin_half_widths[bins],
    yerr=[sigma.error for sigma in sigmas] / bin_centres[bins],
    linestyle="",
    label=r"$\sigma\left(\Delta E\right)$",
    color=colors[1],
)
plt.ylabel(r"$\frac{\sigma\left(\Delta E\right)}{E_\mathrm{reco}}$")
plt.xlabel(r"$E_\mathrm{reco}\;[\mathrm{GeV}]$")
ax = plt.gca()
plt.text(
    0.0,
    1.02,
    "preliminary",
    fontfamily="sans-serif",
    fontsize=16,
    transform=ax.transAxes,
    usetex=False,
)
plt.text(
    0.8,
    1.02,
    "AdvSND",
    fontweight="bold",
    fontfamily="sans-serif",
    fontsize=16,
    transform=ax.transAxes,
    usetex=False,
)
plt.savefig("plots/energy_resolution.pdf")
plt.savefig("plots/energy_resolution.png")